In [12]:
import sys
deriva_py = '/Users/carl/Repos/deriva-py/deriva_common'
identifier = '/Users/carl/Repos/identifiers' 

sys.path
sys.path.extend([deriva_py, identifier])

In [16]:
import pandas as pd
import numpy as np

import synapse_utils
import os

from deriva_common import HatracStore, ErmrestCatalog, get_credential

import IPython.core.debugger

ImportError: cannot import name 'VersionedCatalog'

Set up debugging and logging.

In [3]:
dbg = IPython.core.debugger.Pdb
from IPython.core.debugger import set_trace

# Configuring the logger for debug level will display the uri's generated by the api
debug = False
if debug:
    import logging
    logger = logging.getLogger('deriva_common.datasets')
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    logger.addHandler(ch)

Connect to synapse catalog

In [4]:
# Need to use Deriva authentication agent before executing this
credential = get_credential("synapse.isrd.isi.edu")

objectstore = HatracStore('https','synapse.isrd.isi.edu',credentials=credential)
catalog = ErmrestCatalog('https','synapse.isrd.isi.edu', 1, credentials=credential)

Use Deriva datapath library to construct queries to the synapse catalog and retrieve the information about the current set of studies.

In [5]:
pb = catalog.getPathBuilder()

# convenient name for the schema we care about.
zebrafish = pb.Zebrafish
synapse = pb.Synapse

# Lets get some shortcuts for awkward table names.
study_table = zebrafish.tables['Synaptic Pair Study']
pair_table = zebrafish.tables['Image Pair Study']

# Build up the path study->pair->behavior. 
# Make aliases for study and pair instances for later use.
# We use the left join to make sure we get controls, which will be studies without behaviors.
path = study_table.alias('study')\
    .link(pair_table.alias('pair'))\
    .link(zebrafish.Behavior, join_type = 'left', on=(pair_table.Subject == zebrafish.Behavior.Subject))
    
# Now lets go back an pick up the protocols which are associated with an image.
# Each image has a protocol step, and from the step we can get the protocol.
# Use the first image. 
path = path.pair.link(zebrafish.Image, on=path.pair.columns['Image 1'] == zebrafish.Image.ID)\
    .link(synapse.tables['Protocol Step'])\
    .link(synapse.Protocol)
    
# Now that we have build up the path, we can retrieve the set of studies and associated values
study_entities = path.study.entities(path.study.Study, 
                                     path.pair.Subject, 
                                     path.Behavior.columns['Learned?'],
                                     path.study.columns['Region 1 URL'],
                                     path.study.columns['Region 2 URL'],
                                     Protocol=path.Protocol.ID,)

study_uri = study_entities.uri

print('Identified %d studies' % len(study_entities))

Identified 28 studies


Now lets do some local manipulation of the studies in preperation to turning them into PANDAs.  Specifically, we will map the protocol name into the protocol type.

In [6]:
study_list = []

protocol_types = {
    'PrcDsy20160101A': 'aversion',  
    'PrcDsy20170613A': 'conditioned', 
    'PrcDsy20170615A': 'unconditioned', 
    'PrcDsy20170613B': 'control'
}

for i in study_entities:
    if protocol_types[i['Protocol']] == 'aversion':
        i['Type'] = 'learner' if i['Learned?'] == True else 'nonlearner'
    else:
        i['Type'] = protocol_types[i['Protocol']]
    study_list.append(i)

In [ ]:
study_list[0]

Now lets map over the list of studies and for each study, retrieve the synapse data and turn into PANDA.

In [ ]:
synapses = [synapse_utils.get_synapses(objectstore, study) for study in study_list]

Lets set up to do a 3D scatter plot of the synapse data.

In [ ]:
# Plot a study

import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

def plot_study(study):
    '''
    Create a 3D scatter plot of a study.
    '''
    before_synapses = go.Scatter3d(
        x=study['Before']['X'],
        y=study['Before']['Y'],
        z=study['Before']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.8
        )
    )

    after_synapses = go.Scatter3d(
        x=study['After']['X'],
        y=study['After']['Y'],
        z=study['After']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.8
        )
    )

    data = [before_synapses, after_synapses]

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [ ]:
# Plot synapses from PANDAs....

plot_study(synapses[0])

Use a utility function to dump out all of the studies into a local directory.  
This should eventually be eliminated in favor of just dumping out a BDBag.

In [ ]:
# Dump out the synapses to a local directory

destdir = '/Users/carl/Desktop'
current_dir = os.getcwd()

try:
    os.chdir(destdir)
    synapse_utils.export_synapse_studies(objectstore, study_list)
finally:
    os.chdir(current_dir)


Create a MINID for the data set using the current version of the catalog.

In [18]:
import deriva_minid
import deriva_common.deriva_common.versioned_catalog as vc

vc = vc.VersionedCatalog(study_uri)

config = mca.parse_config(args.config)
server = config['server']
email = config['email']
code = config['code']
test = True

minid = create_catalog_minid(vc, server, email, code, title=args.title, test=test)

print("Created new identifier: %s" % minid)


ModuleNotFoundError: No module named 'versioned_catalog'